<a href="https://colab.research.google.com/github/johyunkang/py_pandas/blob/main/python_ml_guide_03_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01 정확도 (Accuracy)

$ 정확도(accuracy) = \dfrac {예측 결과가 동일한 데이터 건수} {전체 예측 데이터 건수}$



In [ ]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  # fit() 메서드는 아무것도 학습하지 않음
  def fit(self, X, y=None):
    pass

  # predict() 메서드는 단순히 Sex 피처가 남자(1)면 0, 여자(0)면 1로 예측
  def predict(self, X):
    pred = np.zeros((X.shape[0], 1))
    for i in range(X.shape[0]) :
      if(X['Sex'].iloc[i] = 1) :
        pred[i] = 0
      else :
        pred[i] = 1
    return pred

#### MNIST 데이터 세트를 이용하여 값이 7인것만 True, 나머지 숫자는 모두 False로 변환 한 이진 분류 문제

![mnist-binary-classfi](https://user-images.githubusercontent.com/291782/138015598-e638a983-4c43-4df0-af89-36c2c647bcb6.png)


**정확도 평가 지표의 맹점**

In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator) :
    def fit(self, X, y) :
        pass

    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0 값으로 만들어서 반환
    def predict(self, X) :
        return np.zeros((len(X), 1), dtype=bool)

# 사이킷런의 내장 데이터 세트인 load_digits() 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7이면 True이고 이를 astype(int)로 1로 변환
# 7 이 아니면 False 이고 0으로 변환
y = (digits.target == 7).astype(int)
x_train, x_test, y_train, y_test = train_test_split(digits.data, y, random_state = 11)

# 불균형한 레이블 데이터 세트 확인
print('레이블 테스트 세트 크기:', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습 / 예측 / 정확도 평가
fake_clf = MyFakeClassifier()
fake_clf.fit(x_train, y_train)
fake_pred = fake_clf.predict(x_test)
print('\n 모든 예측을 0으로 하여도 정확도는:{0:.3f}'.format(accuracy_score(y_test, fake_pred)))



레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포
0    405
1     45
dtype: int64

 모든 예측을 0으로 하여도 정확도는:0.900


## 02 오차행렬 (Confusion Matrix)

![confusion-matrix-02](https://user-images.githubusercontent.com/291782/138031897-d991171d-369f-4662-900f-1cffacb38d9c.png)


In [4]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fake_pred)

array([[405,   0],
       [ 45,   0]])

위의 표시된 confusion_matrix() 값이 위 이미지와 동일한 위치이다.
- TN : array[0,0]으로 405
- FP : array[0,1] 으로 0
- FN : array[1,0] 으로 45
- TP : array[1,1] 으로 0

정밀도 (Precision) = TP / (TP + FP)

재현율 (Recall) = TP / (TP + FN) = 민감도(Sensitivity) = TPR (True Positive Rate)

재현율이 중요 지표인 경우는 실제 Positive 양성 데이터를 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우
- 예) 암판단 모델 (암:Positive, 암X:Negative) : 암인데 암이 아닌것으로 판단하면 생명을 앗아감 
- 암이 아닌것을 암으로 판단한 경우는 다시 한 번 재검사 하는 수준의 비용만 발생


정밀도가 중요한 지표인 경우 (실제 Negative 음성인 데이터 예측을, Positive 양성으로 잘못 판단하게 되면 업무상 큰 영향)
- 예) 스팸메일 : 실제 Positive인 스팸 메일을 Negative인 일반 메일로 분류하더라도 사용자가 불편을 느끼는 정도지만,
- 실제 Negative 인 일반 메일을 Positive인 스팸 메일로 분류할 경우 메일을 받지 못해 업무에 차질이 생김



In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
# from sklearn.preprocessing import LabelEncoder
import sklearn.preprocessing

def get_clf_eval(y_test, pred) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도(accuracy):{0:.4f}, 정밀도(precision):{1:.4f}, 재현율(recall):{2:.4f}'.format(accuracy, precision, recall))


# null 처리 함수
def fillna(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  df['Fare'].fillna(0, inplace=True)
 
  return df


# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
  df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

  return df


# 레이블 인코딩 수행
def format_features(df):
  df['Cabin'] = df['Cabin'].str[:1] # 앞자리 대문자 1자리만 잘라내기
  features = ['Cabin', 'Sex', 'Embarked']
  for feature in features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])

  return df


# 앞에서 설정한  데이터 전처리 함수 호출
def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_features(df)
  return df

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터 / 테스트 데이터 분할
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/titanic/train.csv')
y_df = df['Survived']
x_df = df.drop('Survived', axis = 1)
x_df = transform_features(x_df)
print(x_df)


x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.2, random_state = 11)

lr = LogisticRegression()

lr.fit(x_train, y_train)
pred = lr.predict(x_test)

print('\n\n## 오차행렬, 정확도, 정밀도, 재현율 등 출력 ##\n')
get_clf_eval(y_test, pred)


     Pclass  Sex        Age  SibSp  Parch     Fare  Cabin  Embarked
0         3    1  22.000000      1      0   7.2500      7         3
1         1    0  38.000000      1      0  71.2833      2         0
2         3    0  26.000000      0      0   7.9250      7         3
3         1    0  35.000000      1      0  53.1000      2         3
4         3    1  35.000000      0      0   8.0500      7         3
..      ...  ...        ...    ...    ...      ...    ...       ...
886       2    1  27.000000      0      0  13.0000      7         3
887       1    0  19.000000      0      0  30.0000      1         3
888       3    0  29.699118      1      2  23.4500      7         3
889       1    1  26.000000      0      0  30.0000      2         0
890       3    1  32.000000      0      0   7.7500      7         2

[891 rows x 8 columns]


## 오차행렬, 정확도, 정밀도, 재현율 등 출력 ##

오차 행렬
[[104  14]
 [ 13  48]]
정확도(accuracy):0.8492, 정밀도(precision):0.7742, 재현율(recall):0.7869


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### 정밀도와 재현율 트레이드 오프(trade-off)

일반적으로 이진 분류에서는 임곗값(예측확률)을 0.5, 즉 50%로 정하고 기준값 보다 크면 Positive, 작으면 Negative로 결정함

**predict_proba()** : 예측 확률 변환 메서드
- 입력 파라미터 : predict() 메서드와 동일하게 보통 테스트 피처 데이터 세트를 입력
- 반환 값
 - 개별 클래스와 예측확률을 ndarray m x n (m:입력 값의 레코드 수, n: 클래스 값 유형) 형태로 반환
 - 입력 테스트 데이터 세트의 표본수가 100개이고 예측 클래스 값 유형이 2개(이진분류)라면 반환 값은 100 x 2 ndarray 임
 - 각 열은 개별 클래스의 예측 확률임. 이진 분류에서 첫 번째 컬럼은 0 Negative 확률, 두번째 컬럼은 1 Positive 확률임

In [23]:
pred_proba = lr.predict_proba(x_test)
pred = lr.predict(x_test)
print('pred() 결과:', pred.shape)
print('pred array에서 앞 3개만 추출:\n', pred[:3])
print('\n')
print('pred_proba() 결과:', pred_proba.shape)
print('pred_proba array에서 앞 3개만 추출:\n', pred_proba[:3])

print('\n')
# 예측 확률 array와 예측 결괏값 array를 병합(concatenate)해 예측 확률과 결괏값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1, 1)], axis = 1)
print('두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])

pred() 결과: (179,)
pred array에서 앞 3개만 추출:
 [1 0 0]


pred_proba() 결과: (179, 2)
pred_proba array에서 앞 3개만 추출:
 [[0.46191519 0.53808481]
 [0.878675   0.121325  ]
 [0.87716185 0.12283815]]


두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 
 [[0.46191519 0.53808481 1.        ]
 [0.878675   0.121325   0.        ]
 [0.87716185 0.12283815 0.        ]]


#### Binarizer 클래스 사용법

In [25]:
from sklearn.preprocessing import Binarizer

X = [[1, -1, 2],
     [2, 0, 0],
     [0, 1.1, 1.2]]

# X의 개별 원소들이 threshold 값보다 같거나 작으면 0을, 크면 1을 반환
binarizer = Binarizer(threshold = 1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
